<a href="https://colab.research.google.com/github/kcalizadeh/PDP_data_processing/blob/master/w2v_for_imports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Mounting Drive

In [1]:
# this cell mounts drive, sets the correct directory, then imports all functions
# and relevant libraries via the functions.py file
from google.colab import drive
import sys

drive.mount('/gdrive',force_remount=True)

drive_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP'

sys.path.append(drive_path)

Mounted at /gdrive


In [2]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser


# a function for quickly testing w2v models
def test_w2v(model, pairs):
  for (pos, neg) in pairs:
    math_result = model.most_similar(positive=pos, negative=neg)
    print(f'Positive - {pos}\tNegative - {neg}')
    [print(f"- {result[0]} ({round(result[1],5)})") for result in math_result[:5]]
    print()

### Load the Data

In [3]:
df = pd.read_csv('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
209037,Difference And Repetition,Deleuze,continental,"Neither contemplative nor active, it is presci...","Neither contemplative nor active, it is presci...",50,"neither contemplative nor active, it is presci...","['neither', 'contemplative', 'nor', 'active', ...","neither contemplative nor active , -PRON- be ..."
85760,Aristotle - Complete Works,Aristotle,aristotle,"Furthermore, I think that you are well aware t...","Furthermore, I think that you are well aware t...",230,"furthermore, i think that you are well aware t...","['furthermore', 'think', 'that', 'you', 'are',...","furthermore , -PRON- think that -PRON- be wel..."
90554,Essay Concerning Human Understanding,Locke,empiricism,"That this is so, and that men in framing diffe...","That this is so, and that men in framing diffe...",454,"that this is so, and that men in framing diffe...","['that', 'this', 'is', 'so', 'and', 'that', 'm...","that this be so , and that man in frame diffe..."
155024,Lewis - Papers,Lewis,analytic,If the histories permitted by a system formed ...,If the histories permitted by a system formed ...,364,if the histories permitted by a system formed ...,"['if', 'the', 'histories', 'permitted', 'by', ...",if the history permit by a system form a tree...
65053,Aristotle - Complete Works,Aristotle,aristotle,When this moisture is heated up by the sun and...,When this moisture is heated up by the sun and...,185,when this moisture is heated up by the sun and...,"['when', 'this', 'moisture', 'is', 'heated', '...",when this moisture be heat up by the sun and ...


In [4]:
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence_str'].map(lambda x: simple_preprocess(x.lower(),deacc=True,
                                                        max_len=100))

In [5]:
# check how it worked
print(df.iloc[290646]['sentence_str'])
df['gensim_tokenized'][290646]

A spider conducts operations that resemble those of a weaver, and a bee puts to shame many an architect in the construction of her cells.


['spider',
 'conducts',
 'operations',
 'that',
 'resemble',
 'those',
 'of',
 'weaver',
 'and',
 'bee',
 'puts',
 'to',
 'shame',
 'many',
 'an',
 'architect',
 'in',
 'the',
 'construction',
 'of',
 'her',
 'cells']

Hmm, an interesting observation. 

### Transfer Learning with GloVe

We'll import GloVe vectors as w2v, then use those as a base from which to train new vectors that are tuned to our corpus.

In [6]:
# load the vectors. other vector sizes were used but yielded generally less sensible models
glove_file = datapath('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [7]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [8]:
# check out how GloVe works on our test pairs
test_w2v(glove_vectors, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- morality (0.82654)
- legal (0.82652)
- laws (0.81529)
- constitutional (0.80616)
- fundamental (0.80217)

Positive - ['self', 'consciousness']	Negative - []
- sense (0.83446)
- mind (0.79755)
- vision (0.78202)
- belief (0.78031)
- life (0.77984)

Positive - ['dialectic']	Negative - []
- hegelian (0.88376)
- dialectical (0.83417)
- dialectics (0.80672)
- materialist (0.77674)
- metaphysics (0.77488)

Positive - ['logic']	Negative - []
- reasoning (0.81405)
- intuitionistic (0.76531)
- concepts (0.75831)
- logical (0.75604)
- theory (0.75026)



Now we want these to be trained on our actual philosophical texts - that way we can see how different thinkers use different words and potentially use the vectors for classification.

So in the cells below we train the existing GloVe model on on the German Idealist texts as a test.

#### German Idealism Example

In [9]:
# isolate the relevant school
documents = df[df['school'] == 'german_idealism']['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=20, threshold=10, delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=20, threshold=10, delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
base_model = Word2Vec(size=300, min_count=5)
base_model.build_vocab(trigramed_tokens)
total_examples = base_model.corpus_count

# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
base_model.train(trigramed_tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv
del base_model

In [10]:
test_w2v(base_model_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- moral law (0.8253)
- rule (0.80809)
- happiness (0.77654)
- morality (0.7688)
- freedom (0.76449)

Positive - ['self', 'consciousness']	Negative - []
- self consciousness (0.91162)
- essence (0.85294)
- immediacy (0.85248)
- activity (0.84101)
- negativity (0.83807)

Positive - ['dialectic']	Negative - []
- antinomy (0.93849)
- doctrine (0.92916)
- procedure (0.9166)
- method (0.90805)
- morals (0.90231)

Positive - ['logic']	Negative - []
- pure reason (0.84827)
- science (0.8375)
- philosophy (0.81935)
- idealism (0.81837)
- doctrine (0.80218)



These seem to reflect the true uses of words in German Idealism. 'Self' + 'consciousness' is rightly associated with 'self consciousness' and 'moral' + 'law' with 'moral law'. It even identifies the German Idealist tendency to unify logic and metaphysics. 

These vectors can be fairly said to reflect how german idealists use these terms. Moreover, they are significantly different than the original GloVe model, which indicates that there was real learning going on here.

For comparison, let's check these same terms, but as used by Phenomenologists.

#### Phenomenology Comparision

In [11]:
def train_glove(source_type, source, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df[source_type] == source]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [12]:
ph_model = train_glove(source_type='school', source='phenomenology', glove_vectors=glove_vectors)

In [13]:
test_w2v(ph_model, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- linguistic (0.99382)
- variety (0.9938)
- empathy (0.99356)
- economic (0.99295)
- overlapping (0.99282)

Positive - ['self', 'consciousness']	Negative - []
- potentiality (0.95791)
- existence (0.94801)
- nature (0.9471)
- future (0.94287)
- authentic (0.94229)

Positive - ['dialectic']	Negative - []
- falling (0.98635)
- poem (0.98557)
- naturally (0.98509)
- sky (0.9836)
- theses (0.98292)

Positive - ['logic']	Negative - []
- definition (0.98299)
- constancy (0.98277)
- correlation (0.98069)
- disclosure (0.98017)
- illumination (0.97769)



Using the phenomenology vectors on some central terms of German idealism once again yields some pretty compelling results, except for where the words are rarely used by the phenomenologists. This is to be expected. Let's try the word vectors on some central terms of phenomenology.

In [14]:
pairs_to_try = [(['perception'], []),
                (['dasein'], []),
                (['consciousness'], []),
                (['method'], []),]

test_w2v(ph_model, pairs_to_try)

Positive - ['perception']	Negative - []
- act (0.93842)
- movement (0.93674)
- care (0.91779)
- reality (0.91767)
- art (0.9072)

Positive - ['dasein']	Negative - []
- being (0.89888)
- itself (0.88563)
- truth (0.83318)
- existence (0.8247)
- consciousness (0.82319)

Positive - ['consciousness']	Negative - []
- future (0.92889)
- death (0.91486)
- movement (0.91095)
- potentiality (0.91071)
- care (0.90202)

Positive - ['method']	Negative - []
- spirit (0.97059)
- value (0.95449)
- necessity (0.94986)
- task (0.94845)
- logic (0.94804)



#### Training on every school & author

To further explore this, we'll train w2v models in this way for each school and examine how each of them looks at the same word - 'philosophy.' We can use these in our future dashboard work.

In [15]:
w2v_dict = {}

for school in df['school'].unique():
  w2v_dict[school] = train_glove('school', school, glove_vectors=glove_vectors)
  print(f'{school} completed')

plato completed
aristotle completed
empiricism completed
rationalism completed
analytic completed
continental completed
phenomenology completed
german_idealism completed
communism completed
capitalism completed
stoicism completed


In [16]:
for school in df['school'].unique():
  print(f'\t{school.upper()}')
  print('----------------------')
  test_w2v(w2v_dict[school], [(['philosophy'], [])])

	PLATO
----------------------
Positive - ['philosophy']	Negative - []
- contrast (0.95035)
- stream (0.94899)
- an eye (0.94669)
- friendship (0.93598)
- greece (0.93358)

	ARISTOTLE
----------------------
Positive - ['philosophy']	Negative - []
- respiration (0.89357)
- studied (0.87402)
- hope (0.87273)
- criticisms (0.87002)
- employing (0.86429)

	EMPIRICISM
----------------------
Positive - ['philosophy']	Negative - []
- religion (0.92729)
- vulgar (0.90778)
- practice (0.90197)
- even (0.88735)
- human nature (0.87892)

	RATIONALISM
----------------------
Positive - ['philosophy']	Negative - []
- doctrine (0.95424)
- name (0.95295)
- church (0.95164)
- person (0.9445)
- approval (0.94252)

	ANALYTIC
----------------------
Positive - ['philosophy']	Negative - []
- philosophical (0.88292)
- semantics (0.87492)
- carnap (0.86052)
- naming (0.84756)
- martin (0.84616)

	CONTINENTAL
----------------------
Positive - ['philosophy']	Negative - []
- unreason (0.95924)
- metaphysics (0.95

Interestingly, many of these top words align quite strongly with the school's general attitude towards philosophy. Continental thinkers mentioning unreason, analytic philosophers focusing on semantics, and phenomenologists associating philosophy with a method all track well. The ones that don't make sense are those that don't problematize the nature of philosophy to any great degree - capitalist thinkers aren't out there trying to discuss the nature of philosophy.

We'd also like vectors trained for each individual author. We can use these in our dashboard to enable intra-school comparisons of authors and generally allow for more fine-grained data exploration.

In [ ]:
#@title Glove Training Function Modified for Authors
def train_glove_author(school, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df['author'] ==school]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [17]:
for author in df['author'].unique():
  w2v_dict[author] = train_glove('author', author, glove_vectors=glove_vectors)
  print(f'{author} completed')

Plato completed
Aristotle completed
Locke completed
Hume completed
Berkeley completed
Spinoza completed
Leibniz completed
Descartes completed
Malebranche completed
Russell completed
Moore completed
Wittgenstein completed
Lewis completed
Quine completed
Popper completed
Kripke completed
Foucault completed
Derrida completed
Deleuze completed
Merleau-Ponty completed
Husserl completed
Heidegger completed
Kant completed
Fichte completed
Hegel completed
Marx completed
Lenin completed
Smith completed
Ricardo completed
Keynes completed
Epictetus completed
Marcus Aurelius completed


With this finished - our next step is to train one on the entire corpus for use in classification.

#### Building a Model for the full Corpus

In [18]:
documents = df['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=30, threshold=10, 
                  delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=30, threshold=10, 
                  delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
all_text_model = Word2Vec(size=300, min_count=5)
all_text_model.build_vocab(trigramed_tokens)
total_examples = all_text_model.corpus_count

# add GloVe's vocabulary & weights
all_text_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
all_text_model.train(trigramed_tokens, total_examples=total_examples, 
                     epochs=all_text_model.epochs)
all_text_wv = all_text_model.wv

As a test case, let's see how the philosophy thinks of itself as compared to how glove thinks of philosophy.

In [19]:
for model in [1, 2]:
  if model == 1:
    print(f'\tPHILOSOPHY CORPUS')
    print('------------------------------------')
    test_w2v(all_text_wv, [(['philosophy'], [])])
  if model == 2:
    print(f'\tBASE GLOVE')
    print('------------------------------------')
    test_w2v(glove_vectors, [(['philosophy'], [])])


	PHILOSOPHY CORPUS
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.78726)
- metaphysics (0.77321)
- science (0.76454)
- religion (0.73403)
- psychology (0.73401)

	BASE GLOVE
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.88151)
- philosophical (0.84362)
- mathematics (0.83389)
- psychology (0.82387)
- sociology (0.81085)



This sort of stands to reason - 'metaphysics' often has a different meaning outside of philosophical discussion, so it's not surprising to see it as the most changed term here. 

#### Exporting for Only New Texts


In [26]:
# use this cell to build the newest author/text/school in the corpus, then export in the cell below
epictetus_wv = train_glove(source_type='author', source='Epictetus', glove_vectors=glove_vectors)
aurelius_wv = train_glove(source_type='author', source='Marcus Aurelius', glove_vectors=glove_vectors)
stoicism_wv = train_glove(source_type='school', source='stoicism', glove_vectors=glove_vectors)

In [33]:
# test out new model
stoicism_wv.most_similar('power')

[('own', 0.9999139308929443),
 ('our', 0.999912679195404),
 ('within', 0.9999122619628906),
 ('natural', 0.9999038577079773),
 ('your', 0.9999033212661743),
 ('life', 0.9999023675918579),
 ('her', 0.9999022483825684),
 ('its', 0.9998999834060669),
 ('take', 0.9998998641967773),
 ('us', 0.9998985528945923)]

In [ ]:
epictetus_wv.save('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/Epictetus_w2v.wordvectors')
stoicism_wv.save('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/Stoiticism_w2v.wordvectors')

#### Finalized exporting

All in all, things look good, so let's export the vectors so that they can be used in our neural networks and in our dash app. 

In [ ]:
# do not run these cells if you want to keep old w2v versions
all_text_wv.save_word2vec_format('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/w2v_for_nn.bin')
all_text_wv.save('/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/w2v_for_nn.wordvectors')

for source in w2v_dict.keys():
  w2v_dict[source].save(f'/gdrive/MyDrive/Colab_Projects/philosophy_data_project/w2v_models/{source}_w2v.wordvectors')

And that's it! See our other notebooks for more of the modeling work. 